## Ligand Receptor Interaction Survival Analysis 

By Tiago Maié and James Nagai

Here we introduce the survival model analysis used to the orthogonal validation of the scACCorDiON subcluters.

### For installing the R package use
```{r}
pak::pak("CostaLab/scACCorDiON.su")
```

In [1]:
library(scACCorDiON.su)
library(IRdisplay) ## Tables rendering 
library(reactable) ## Tables display


---------------------------------
scACCorDiON.su version 0.0.0.9000
---------------------------------



### Loading the data used in the PDAC experiment

In [2]:
data(pdac_cci_data)
data(paad_tcga_clinical_data) ## TCGA data
data(paad_tcga_expression_data) ## TCGA data
data(comparison_of_interest) ## Here he have the desired interaction

### Interaction: DCT2 and DCT1 were renamed in the paper for the readability, Malignant Ductal and Ductal cell, respectivelly

In [3]:
comparison_of_interest

[1] "Ductal.cell.type.2_Ductal.cell.type.1"

### Reading CrossTalkeR sankey data (Supp. Fig. 8)

Here we can supply ligand receptor interactions for the PDAC data

In [4]:
lrsankey <- read.csv("../../../scaccordion/data/PDAC_differential.csv")

*Note:* table must be names as the example below

In [5]:
head(lrsankey[,c("source","target","gene_A","gene_B","LRScore")]) 

,source,target,gene_A,gene_B,LRScore
,<chr>,<chr>,<chr>,<chr>,<dbl>
1,Ductal cell type 2,Macrophage cell,B2M|L,CD1C|R,-2.578362
2,Endocrine cell,B cell,INS|L,LILRB1|R,-2.548423
3,Ductal cell type 2,B cell,B2M|L,CD1C|R,-2.495319
4,B cell,B cell,APP|L,CD74|R,-2.422047
5,B cell,B cell,COPA|L,CD74|R,-2.386160
6,T cell,B cell,COPA|L,CD74|R,-2.381791


## Preparing the data from the sankey plot to perform survival analysis(Table 4)

Here, the data is prepared to be used in the **run_survival** funtion. We first select the source and target interaction. Second, HLA genes are filtered. Finally, we filter the overlapping communication gened and select the top 10

In [6]:
pairs <- lrsankey |>
            dplyr::filter(grepl("Ductal cell type 2",source))|>
            dplyr::filter(grepl("Ductal cell type 1",target)) |>
            dplyr::filter(!grepl("HLA-",gene_A)) |>
            dplyr::mutate(ligand=gsub("\\|L","",gene_A)) |>
            dplyr::mutate(receptor=gsub("\\|R","",gene_B)) |>
        	dplyr::filter(ligand %in% paad_tcga_expression_data$symbol) |>
        	dplyr::filter(receptor %in% paad_tcga_expression_data$symbol) |>
            dplyr::slice_max(n=10,order_by = LRScore,with_ties = FALSE) |>
            dplyr::select(ligand,receptor)

Once this analysis were dont comparing PDAC2_x_PDAC1 the highest LRscores are associated to interactions that are highly expressed in PDAC1

### LR Survival

In [7]:
results <-
  run_survival(
    comparison_oi = comparison_of_interest,
    lr_data = pdac_cci_data,
    tcga_exp_data = paad_tcga_expression_data,
    tcga_clinical_data = paad_tcga_clinical_data,
    selection_method = c("limma"),
    custom_selection = pairs,
    n_lr_selected = 10,
    which_get = c("ligand", "receptor"),
    is_signif = TRUE,
    clinical_vars_in_model = c("stage")
  )
#> Warning: Zero sample variances detected, have been offset away from zero
results$cox_models$coxph_clinical.lr_pairs |>
  broom::tidy() |>
  dplyr::mutate(issignif=p.value<0.05) |>
  reactable()

HTML widgets cannot be represented in plain text (need html)

### Ligand only survival

In [8]:
results <-
  run_survival(
    comparison_oi = comparison_of_interest,
    lr_data = pdac_cci_data,
    tcga_exp_data = paad_tcga_expression_data,
    tcga_clinical_data = paad_tcga_clinical_data,
    selection_method = c("limma"),
    custom_selection = pairs,
    n_lr_selected = 10,
    which_get = c("ligand"),
    is_signif = TRUE,
    clinical_vars_in_model = c("stage")
  )
#> Warning: Zero sample variances detected, have been offset away from zero
results$cox_models$coxph_clinical.gene_expression |>
  broom::tidy() |>
  dplyr::mutate(issignif=p.value<0.05) |>
  reactable()

HTML widgets cannot be represented in plain text (need html)

### Receptor only survival

In [9]:
results <-
  run_survival(
    comparison_oi = comparison_of_interest,
    lr_data = pdac_cci_data,
    tcga_exp_data = paad_tcga_expression_data,
    tcga_clinical_data = paad_tcga_clinical_data,
    selection_method = c("limma"),
    custom_selection = pairs,
    n_lr_selected = 10,
    which_get = c("receptor"),
    is_signif = TRUE,
    clinical_vars_in_model = c("stage")
  )
#> Warning: Zero sample variances detected, have been offset away from zero
results$cox_models$coxph_clinical.gene_expression |>
  broom::tidy() |>
  dplyr::mutate(issignif=p.value<0.05)|>
  reactable()

HTML widgets cannot be represented in plain text (need html)

## Finding survival hits without custom Selection

In [13]:
results <-
  run_survival(
    comparison_oi = comparison_of_interest,
    lr_data = pdac_cci_data,
    tcga_exp_data = paad_tcga_expression_data,
    tcga_clinical_data = paad_tcga_clinical_data,
    selection_method = c("limma"),
    n_lr_selected = 30,
    which_get = c("ligand", "receptor"),
    is_signif = TRUE,
    clinical_vars_in_model = c("stage")
  )
#> Warning: Zero sample variances detected, have been offset away from zero
results$cox_models$coxph_clinical.lr_pairs |>
  broom::tidy() |>
  dplyr::mutate(issignif=p.value<0.05) |>
  reactable()

Warning message:
“Zero sample variances detected, have been offset away from zero”


HTML widgets cannot be represented in plain text (need html)